In [1]:
%load_ext autoreload
%autoreload 2

This notebook will closely follow the [Simulating Molecules using VQE Qiskit Tutorial](https://qiskit.org/textbook/ch-applications/vqe-molecules.html). 
$$\newcommand{\ket}[1]{|{#1}\rangle}\newcommand{\bra}[1]{\langle{#1}|}$$

## Two Qubit VQE

Our goal is to estimate the ground state energy $\lambda_{\text{min}}$ of some Hamiltonian $H$. 

By the variational principle, we know the below is true for some quantum state $|\psi (\vec{\theta})\rangle$ of norm 1, where $\vec{\theta}$ represent parameters that we can set to realize any valid quantum state.

$$ \begin{align}\langle \psi(\vec{\theta})  | H | \psi(\vec{\theta}) \rangle &= \sum_{\lambda_i,\lambda_j} \langle \psi(\vec{\theta}) | \psi_{\lambda_i} \rangle \langle \psi_{\lambda_i} | H | \psi_{\lambda_j} \rangle \langle \psi_{\lambda_j} | \psi(\vec{\theta}) \rangle \\
&= \sum_{\lambda} \lambda |\langle \psi_{\lambda} | \psi(\vec{\theta}) \rangle |^2 \\
&\geq \lambda_{\text{min}} \underbrace{\sum_{\lambda}  |\langle \psi_{\lambda} | \psi(\vec{\theta}) \rangle|^2}_{1} = \lambda_{\text{min}}\end{align}$$ 

We also know that for $|\psi (\vec{\theta_{\text{min}}})\rangle = |\psi_{\text{min}}\rangle$ (the ground state eigenfunction), we satisy the above inequality. 

Thus, we must
1. Find some way to construct an eigenfunction $|\psi (\vec{\theta})\rangle$.
2. Optimize over the $\vec{\theta}$ to minimize $\langle \psi(\vec{\theta})  | H | \psi(\vec{\theta}) \rangle$ and find the lowest upper bound to $\lambda_{\text{min}}$. 

With this motivation in mind, let's begin with the simplest case of a 1 qubit Hilbert space. Here, we can access any arbitrary quantum state by applying the $U_3$ gate on an arbitrary intial state $\ket{\psi_0}$, namely $\ket{\psi (\vec{\theta})} = U_3(\theta_1,\theta_2, \theta_3)\ket{\psi_0}$ where 

$$\begin{align}U_3(\vec{\theta}) &= \begin{pmatrix} \cos\left(\frac{\theta_1}{2}\right) & -e^{i \theta_3} \sin\left(\frac{\theta_1}{2}\right)\\e^{i\theta_2}\sin\left(\frac{\theta_1}{2}\right)& e^{i\theta_3 + i\theta_2}\cos\left(\frac{\theta_1}{2}\right)\end{pmatrix}\end{align}$$

Now that we have defined $\ket{\psi (\vec{\theta})}$ we can minimize the $\langle \psi(\vec{\theta})  | H | \psi(\vec{\theta}) \rangle$ matrix over $\vec{\theta}$. The classical optimizer we will use to do this will first use the _Constrained Optimization by Linear Approximation optimizer_ (COBYLA) optimizer in the noiseless case, and the _Simultaneous Perturbation Stochastic Approximation_ (SPSA) optimizer for a noisy simulator. 

### Example 
#### _(ground state energy of random single qubit Hermitian operator)_

##### measuring < psi | H | psi>

To orient the reader, we begin by demonstrating how to find $\langle \psi | H | \psi \rangle$ using Qiskit. While one could simply take the matematical overlap, this approach quickly becomes computationally inefficient for larger systems. Instead, we estimate $\langle \psi | H | \psi \rangle$ using multiple shots of an executed quantum circuit. In the example below, we take the expectation of the $H_2$ hydrogen molecule hamiltonian against the $\ket{0}$ qubit state. 

In [2]:
from qiskit import Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.operators import PauliExpectation, CircuitStateFn, CircuitSampler, StateFn
from qiskit.aqua.operators import MatrixExpectation, AerPauliExpectation
from qiskit.aqua.operators import X, Y, Z, I
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# H2-molecule Hamiltonian 
op =  (-1.0523732 * I^I) + (0.39793742 * I^Z) + (-0.3979374 * Z^I) \
    + (-0.0112801 * Z^Z) + (0.18093119 * X^X)

In [4]:
# define the state you w.r.t. which you want the expectation value
psi = QuantumCircuit(2)
# convert to a state
psi = CircuitStateFn(psi)

In [5]:
# "Mathematical" < psi | O | psi > 
print('Math:', psi.adjoint().compose(op).compose(psi).eval().real)

Math: -1.06365328


In [6]:
# Shot-based Results

backend = Aer.get_backend('qasm_simulator') 
q_instance = QuantumInstance(backend, shots=1024)

# define the state to sample
measurable_expression = StateFn(op, is_measurement=True).compose(psi) 

# convert to expectation value
expectations = {}
expectations['shots'] = PauliExpectation().convert(measurable_expression)
expectations['aer'] = AerPauliExpectation().convert(measurable_expression)
expectations['matrix'] = MatrixExpectation().convert(measurable_expression)

samplers = {}
for label, expectation  in expectations.items():
    samplers[label] = CircuitSampler(q_instance).convert(expectations[label]) 

# evaluate
for label, sampler  in samplers.items():
    print(label + ": ", sampler.eval().real)  

shots:  -1.0526984618554687
aer:  -1.06365328
matrix:  -1.06365328


And so, we see that `AerPauliExpectation` and `MatrixExpectation` are exact methods, while `PauliExpectation` uses shots. 

##### optimization using 1 qubit

Here, we define a random Hermitian operator $H$ within a 1-qubit Hilbert Space and use the variational principle to find the ground state energy `lambda_min` of $H$. 

In [7]:
backend = Aer.get_backend('qasm_simulator') 
q_instance = QuantumInstance(backend, shots=1024)

# setup random hermitian 
np.random.seed(0)
rand_coeff = np.random.randn(4)
op = rand_coeff[0]*I + rand_coeff[1]*X + rand_coeff[2]*Y + rand_coeff[3]*Z
op_np = op.to_matrix()

def get_measurable_expression(params):
    qr = QuantumRegister(1, name="q")
    psi_qc = QuantumCircuit(qr)
    psi_qc.u(params[0], params[1], params[2], qr[0])
    psi_qc = CircuitStateFn(psi_qc)
    return StateFn(op, is_measurement=True).compose(psi_qc) 
    
def objective_function(params):
    # Obtain a quantum circuit instance from the paramters
    measurable_expression = get_measurable_expression(params)
    expectation = AerPauliExpectation().convert(measurable_expression)
    sampler = CircuitSampler(q_instance).convert(expectation)
    return sampler.eval().real

print(objective_function([0.1,.1,2]))

4.04325469422588


In [8]:
from qiskit.aqua.components.optimizers import COBYLA

# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500, tol=0.0001)

# Create the initial parameters (noting that our single qubit variational form has 3 parameters)
params = np.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)


# Obtain the output distribution using the final parameters
lambda_min = objective_function(ret[0])


print("Parameters Foussnd: ", ret[0])
print("Minimum Energy (vqe): ", lambda_min)

v,w = np.linalg.eigh(op_np)
print("Minumum Energy (exact): ", v[0])

print("Percent difference: ", (lambda_min - v[0])/np.abs(v[0])*100)

Parameters Found:  [3.58239678 1.18263085 0.24672815]
Minimum Energy (vqe):  -0.7137806032241256
Minumum Energy (exact):  -0.7137806111073199
Percent difference:  1.1044281950047938e-06


The VQE algorithm finds a pretty reasonable (~$10^{-6}$% above exact eigenvalue) ground state energy as desired!

### Example 
#### _(ground state energy of two Qubit H$_2$ Hydrogen Molecule)_

Here, we define a the H$_2$ Hydrogen Molecule hamiltonian $H$ within a 2-qubit Hilbert Space and use the variational principle to find the ground state energy `lambda_min` of $H$. 

In [18]:
backend = Aer.get_backend('qasm_simulator') 
q_instance = QuantumInstance(backend, shots=1024)

# setup random hermitian 
np.random.seed(0)
rand_coeff = np.random.randn(16)
op =  (-1.0523732 * I^I) + (0.39793742 * I^Z) + (-0.3979374 * Z^I) \
    + (-0.0112801 * Z^Z) + (0.18093119 * X^X)

op_np = op.to_matrix()

def get_measurable_expression(params, plot=False):
    qr = QuantumRegister(2, name="q")
    psi_qc = QuantumCircuit(qr)
    psi_qc.u(params[0], params[1], params[2], qr[0])
    psi_qc.u(params[3], params[4], params[5], qr[1])
    psi_qc.cx(qr[1], qr[0])
    
    psi_qc.u(params[6], params[7], params[8], qr[0])
    psi_qc.u(params[9], params[10], params[11], qr[1])
    psi_qc.cx(qr[0], qr[1])
    
    psi_qc.u(params[12], params[13], params[14], qr[0])
    psi_qc.u(params[15], params[16], params[17], qr[1])
    psi_qc.cx(qr[1], qr[0])
    
    psi_qc.u(params[18], params[19], params[20], qr[0])
    psi_qc.u(params[21], params[22], params[23], qr[1])
    
    if plot:
        print(psi_qc)
    psi_qc = CircuitStateFn(psi_qc)
    return StateFn(op, is_measurement=True).compose(psi_qc) 
    
def objective_function(params, plot=False):
    # Obtain a quantum circuit instance from the paramters
    measurable_expression = get_measurable_expression(params, plot=plot)
    expectation = AerPauliExpectation().convert(measurable_expression)
    sampler = CircuitSampler(q_instance).convert(expectation)
    return sampler.eval().real

print(objective_function(np.random.randn(24), plot=True))

     ┌────────────────────────────┐┌───┐ ┌────────────────────────────┐      »
q_0: ┤ U(1.4941,-0.20516,0.31307) ├┤ X ├─┤ U(0.86444,-0.74217,2.2698) ├───■──»
     ├───────────────────────────┬┘└─┬─┘┌┴────────────────────────────┴┐┌─┴─┐»
q_1: ┤ U(-0.8541,-2.553,0.65362) ├───■──┤ U(-1.4544,0.045759,-0.18718) ├┤ X ├»
     └───────────────────────────┘      └──────────────────────────────┘└───┘»
«       ┌──────────────────────────┐ ┌───┐┌────────────────────────────┐
«q_0: ──┤ U(1.5328,1.4694,0.15495) ├─┤ X ├┤ U(-0.34791,0.15635,1.2303) ├
«     ┌─┴──────────────────────────┴┐└─┬─┘├────────────────────────────┤
«q_1: ┤ U(0.37816,-0.88779,-1.9808) ├──■──┤ U(1.2024,-0.38733,-0.3023) ├
«     └─────────────────────────────┘     └────────────────────────────┘
-0.7172682562928445


In [20]:
from qiskit.aqua.components.optimizers import COBYLA

# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500, tol=0.0001)

# Create the initial parameters (noting that our single qubit variational form has 3 parameters)
params = np.random.rand(24)
ret = optimizer.optimize(num_vars=24, objective_function=objective_function, initial_point=params)


# Obtain the output distribution using the final parameters
lambda_min = objective_function(ret[0])


# print("Parameters Found: ", ret[0])
print("Ground State Energy (vqe): ", lambda_min)

v,w = np.linalg.eigh(op_np)
print("Ground State Energy (exact): ", v[0])

print("Percent difference: ", (lambda_min - v[0])/np.abs(v[0])*100)

Ground State Energy (vqe):  -1.8572749553229753
Ground State Energy (exact):  -1.8572749575690393
Percent difference:  1.209333056638081e-07
